# Assignment #9 - Data Gathering and Warehousing - DSSA-5102

Instructor: Melissa Laurino</br>
Spring 2024</br>

Name: Louise Ramos
</br>
Date: April 4, 2024
<br>
<br>
**At this time in the semester:** <br>
--We have explored a dataset. <br>
--We have cleaned our dataset. <br>
--We created a Github account with a repository for this class and included a Metadata file about our data. <br>
--We introduced general and more advanced SQL syntax, queries, and applications in R.<br>
--Created our own database from scratch in R using libraries RMariaDB and RSQLite <br>
--Populated our databases we created from scratch
--Created a visual enhanced entity relationship diagram for our database
<br>

At this point, we have discussed all major statements used with SQL, but the possibilities are endless when it comes to data! Below we will explore some miscellaneous statements and tools that may be useful with your database.<br>

<br>

Read Chapter 7 & 10 in Getting Started with SQL by Thomas Nield available on Blackboard. <br>
A quick reference for SQL commands: https://www.w3schools.com/sql/default.asp <br>

Review the powerpoint and other readings specified on Blackboard in the Discussion Board.<br>

In the event your database does not meet the requirements below to answer the question, please use the database provided in Assignment #4 and #5. Remember to credit your data source.

We used two R packages to create two separate databases for practice in our previous assignment. Feel free to use your preferred package for the exploration below. Now that all of our data is loaded onto the MySQL Workbench server, you can even take the assignment a step further and complete it all within SQL without Jupyter Notebook/RMD/Visual Studio Code.<br>

Follow the instructions below to complete the assignment. Be sure to comment **all** code and answer **all** questions in markdown for full credit. The cells below are the minimum requirement, but exploring larger queries are encouraged.<br>

**Data origin:**


In [132]:
# Importing libraries
import pymysql
import pandas as pd
import sqlite3
import mysql.connector

In [96]:
# Connecting to database
mydb = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = '1734680nN',
    database = 'police_shootings'
)

In [97]:
# Creating a cursor 
cursor = mydb.cursor()

# SQL query to show table names
query = "SHOW TABLES"

# Running the query
cursor.execute(query)
tables = cursor.fetchall()

# Printing the table names
print("Tables")
for table in tables:
    print(table[0])

Tables
AnotherTable
Extra
Incident
Location
Person


In [98]:
# Listing columns in each table
# Creating a cursor 
cursor = mydb.cursor()

# Listing table names
table_names = ['Incident', 'Location', 'Person']
for table_name in table_names:
    print(f"Columns in table '{table_name}':")

    # Retrieving column names for the current table
    query_columns = f"SHOW COLUMNS FROM {table_name}"
    cursor.execute(query_columns)
    columns = cursor.fetchall()

    # Printing the column names
    for column in columns:
        print(column[0])  

# Closing the cursor
cursor.close()

Columns in table 'Incident':
id
person_id
location_id
date
manner_of_death
armed
signs_of_mental_illness
threat_level
flee
body_camera
Columns in table 'Location':
id
city
state
longitude
latitude
is_geocoding_exact
Columns in table 'Person':
id
name
age
gender
race


True

In [99]:
# Testing to see if the tables are populated on the server
# Creating a cursor 
cursor = mydb.cursor()

# Listing the table names
table_names = ['Incident', 'Location', 'Person']
for table_name in table_names:
    print(f"\nTesting data in table '{table_name}':")

    # Query to fetch data from the tables
    query = f"SELECT * FROM {table_name} LIMIT 5"  
    cursor.execute(query)
    records = cursor.fetchall()

    # Displaying the data
    for record in records:
        print(record)

# Closing the cursor
cursor.close()


Testing data in table 'Incident':
(1, None, None, datetime.date(2015, 1, 2), 'shot', 'gun', 1, 'attack', 'Not fleeing', 0)
(2, None, None, datetime.date(2015, 1, 2), 'shot', 'gun', 0, 'attack', 'Not fleeing', 0)
(3, None, None, datetime.date(2015, 1, 3), 'shot and Tasered', 'unarmed', 0, 'other', 'Not fleeing', 0)
(4, None, None, datetime.date(2015, 1, 4), 'shot', 'toy weapon', 1, 'attack', 'Not fleeing', 0)
(5, None, None, datetime.date(2015, 1, 4), 'shot', 'nail gun', 0, 'attack', 'Not fleeing', 0)

Testing data in table 'Location':
(1, 'Shelton', 'WA', Decimal('-123.122000'), Decimal('47.247000'), 1)
(2, 'Aloha', 'OR', Decimal('-122.892000'), Decimal('45.487000'), 1)
(3, 'Wichita', 'KS', Decimal('-97.281000'), Decimal('37.695000'), 1)
(4, 'San Francisco', 'CA', Decimal('-122.422000'), Decimal('37.763000'), 1)
(5, 'Evans', 'CO', Decimal('-104.692000'), Decimal('40.384000'), 1)

Testing data in table 'Person':
(1, 'Tim Elliot', 53, 'M', 'A')
(2, 'Lewis Lee Lembke', 47, 'M', 'W')
(3, 

True

#### CASE Statements
Case statements are similar to the if-then-else statements in programming. The data that meets the case statements in the database will be returned. You start a CASE statement with the word CASE and conclude it with an END. Between those keywords, you specify each condition with a WHEN [condition] THEN [value], where the condition and the values are specified by you.

Nield, Thomas. Getting Started with SQL (p. 71). O'Reilly Media. Kindle Edition. <br>
<br>
Write your question you are answering with your data query. <br>
<br>
**Example Question:** Seeing what incidents happened in Alaska?
<br>
**What tables are we joining?:** Location, Person and Incident

In [118]:
#Write a query that includes a CASE statement and JOIN your results between tables that answer your question.
# Establish a connection to your MySQL database
# Connecting to database
host = 'localhost'
user = 'root'
password = '1734680nN'
database = 'police_shootings'

# Defining the state
state_of_interest = 'AK'

try:
    # Establishing a connection to database
    conn = pymysql.connect(host=host, user=user, password=password, database=database)
    cursor = conn.cursor()

    # SQL query to retrieve names of person involved in incidents in the specified state
    sql_query = f"""
        SELECT
            P.name
        FROM
            Person AS P
        JOIN
            Incident AS I ON P.id = I.person_id
        JOIN
            Location AS L ON I.location_id = L.id
        WHERE
            L.state = '{state_of_interest}';
    """

    # Executing the SQL query
    cursor.execute(sql_query)
    rows = cursor.fetchall()

    # Printing the people involved 
    if rows:
        print(f"Persons involved in incidents in the state of {state_of_interest}:")
        for row in rows:
            person_name = row[0]
            print(person_name)
    else:
        print(f"No persons found involved in incidents in the state of {state_of_interest}")

except pymysql.Error as e:
    print(f"Error: {e}")

No persons found involved in incidents in the state of AK


**CASE STATEMENT**
<br>
Write a second CASE statement!

Write your question you are answering with your data query. <br>
<br>
**Question:** Who is considered a child vs an adult?
<br>
**What tables are we joining?:**  Person

In [125]:
#Write a query that includes a CASE statement and JOIN your results between tables that answer your question.

# Database connection
host = 'localhost'
user = 'root'
password = '1734680nN'
database = 'police_shootings'

try:
    # Establishing a connection
    conn = pymysql.connect(host=host, user=user, password=password, database=database)
    cursor = conn.cursor()

    # SQL query 
    sql_query = """
        SELECT
            name,
            age,
            CASE
                WHEN age < 18 THEN 'Child'
                WHEN age >= 18 AND age < 65 THEN 'Adult'
                ELSE 'Elderly'
            END AS age_group
        FROM
            Person
            LIMIT 5;
    """

    # Executinf the SQL query
    cursor.execute(sql_query)
    rows = cursor.fetchall()

    # Printing the names, ages, and age groups of persons based on the CASE statement
    if rows:
        print("Names and age groups:")
        for row in rows:
            person_name, person_age, age_group = row
            print(f"Name: {person_name}, Age: {person_age}, Age Group: {age_group}")
    else:
        print("No persons found")

except pymysql.Error as e:
    print(f"Error: {e}")


Names and age groups:
Name: Tim Elliot, Age: 53, Age Group: Adult
Name: Lewis Lee Lembke, Age: 47, Age Group: Adult
Name: John Paul Quintero, Age: 23, Age Group: Adult
Name: Matthew Hoffman, Age: 32, Age Group: Adult
Name: Michael Rodriguez, Age: 39, Age Group: Adult


**CASE STATEMENT**
<br>
Write a third CASE statement!

Write your question you are answering with your data query. <br>
<br>
**Question:** Who was in WA when it happened?
<br>
**What tables are we joining?:** Person and Location

In [149]:
#Write a query that includes a CASE statement and JOIN your results between tables that answer your question.
# Database connection 
def retrieve_persons_with_age_groups(host, user, password, database):
    try:
        # Connecting to the database
        conn = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )

        cursor = conn.cursor()

        # Define the SQL query with JOIN, CASE statement for age groups and state
        sql_query = """
            SELECT p.name,
                   l.state,
                   CASE
                       WHEN l.state WA THEN WA
                   END AS state,
                   CASE
                       WHEN l.state = 'WA' THEN 'Washington'
                   END AS state_description
            FROM Person p
            JOIN Location l ON p.id = l.person_id
        """

        # Executin the SQL query
        cursor.execute(sql_query)
        results = cursor.fetchall()

        # Printing the results
        if results:
            print("Results:")
            for row in results:
                name, age, state, age_group, state_description = row
                print(f"Name: {name}, Age: {age}, State: {state}, Age Group: {age_group}, State Description: {state_description}")
        else:
            print("No results found.")

        # Closing the connection
        conn.close()

    except mysql.connector.Error as e:
        print("Error:", e)

# Specifying the MySQL database connection details
host = 'localhost'
user = 'root'
password = '1734680nN'
database = 'police_shootings'

Results:
Tim Elliot, WA
Lewis Lee Lembke, WA
John Paul Quintero, WA
Matthew Hoffman, WA
Michael Rodriguez, WA
Kenneth Joe Brown, WA
Kenneth Arnold Buck, WA
Brock Nichols, WA
Autumn Steele, WA
Leslie Sapp III, WA


**NULL**
<br>
As with all data, NULL values are fields with no data. Null data can be useful to know with the INSERT INTO statement below.

Find the NULL data within your database. Write your question you are answering with your data query. <br>
<br>
**Example Question:**  Which people are missing Age? (Limited to 5)
<br>

In [59]:
#Replace the Example Question with your own and query the data for your question.
# Connecting to the database
conn = pymysql.connect(host='localhost', user='root', password='1734680nN', database='police_shootings')

# Creating a cursor object
cursor = conn.cursor()

# Executing SQL query to select rows with null age in the Person table
query = "SELECT * FROM Person WHERE age IS NULL LIMIT 5"

# Executing the query
cursor.execute(query)
rows_with_null_age = cursor.fetchall()

# Printing the results
print("Rows with NULL age:")
for row in rows_with_null_age:
    print(row)


Rows with NULL age:
(5198, 'Alejandro Salazar', None, 'M', 'H')
(5840, 'Jason Day', None, 'M', 'B')
(5880, 'John Tozzi', None, 'M', None)
(5890, 'Deaunte Lamar Bell', None, 'M', 'B')
(6003, None, None, 'M', None)


**INSERT INTO**
<br>
You can insert new records into a table as needed using the INSERT INTO statement. If you choose to populate a table with certain records and not others, the rest of the fields will remain empty/NULL.
<br>
For INSERT INTO, we are not querying the database, instead we are ADDING to it. We do not need to use dbGetQuery(), but instead, dbExecute()!
<br><br>
**Objective:** Add people into the Person table
<br>
**What table(s) are we adding a record to?** added people into the table

In [91]:
#Use INSERT INTO to manually insert data. 
#Query to insert a person with id, name, gender and race
insert = """
INSERT INTO Person (id, name, age, gender, race)
VALUES 
    (2000, "Josh", 29, M, W),
    (2001, "Sarah", 24, F, H);"""



#It's okay if you do not execute your code if you do not want to add any data to your database.

**MIN() and MAX()**
<br>
You can use these statements alone or in combination with the CASE statemts above. Find the minimum and maximum of a numerical column.
<br><br>
**Question:** Whats the youngest and oldest age in the data?
<br>
**What table(s) are we analyzing?** Person

In [61]:
#Create a simple query to find the MIN() of a value and the MAX() of a value()
#They do not have to be in the same query yet.

# Creating a cursor object
cursor = conn.cursor()

# Querying to calculate MIN and MAX of age
query = """
    SELECT 
        MIN(CASE WHEN age IS NOT NULL THEN age ELSE NULL END) AS min_age,
        MAX(CASE WHEN age IS NOT NULL THEN age ELSE NULL END) AS max_age
    FROM 
        Person;
"""

# Executing the query
cursor.execute(query)
result = cursor.fetchone()

# Printing the result
print("Minimum Age (excluding NULL):", result[0])
print("Maximum Age (excluding NULL):", result[1])



Minimum Age (excluding NULL): 2
Maximum Age (excluding NULL): 92


Combine CASE statement with Min() and Max() for a more detailed query of your data:
<br><br>
**Question:** What are the minimum and maximum ages for different age groups (specifically 'Under 18' and '18-100')?
<br>
**What table(s) are we adding a record to?** Made new table call Extra

In [69]:
#Combine the CASE statement with MIN() and MAX() for a more detailed query of your data.
#You can use one of the same CASE statements as above, or create a new one that relates to your MIN and MAX.

try:
    # connecting database
    conn = pymysql.connect(host='localhost',
                           user='root',
                           password='1734680nN',
                           database='police_shootings')

    # Creating a cursor 
    cursor = conn.cursor()

    # Creating Extra table
    create_table_query = """
        CREATE TABLE IF NOT EXISTS Extra (
            id INT AUTO_INCREMENT PRIMARY KEY,
            age_group VARCHAR(20) NOT NULL,
            min_age INT,
            max_age INT
        );
    """
    cursor.execute(create_table_query)
    print("Extra table created successfully.")

    # Defining SQL query 
    sql_query = """
        SELECT
            MIN(CASE WHEN age < 18 THEN age END) AS min_age_under_18,
            MAX(CASE WHEN age < 18 THEN age END) AS max_age_under_18,
            MIN(CASE WHEN age >= 18 AND age <= 100 THEN age END) AS min_age_18_100,
            MAX(CASE WHEN age >= 18 AND age <= 100 THEN age END) AS max_age_18_100
        FROM
            Person;
    """

    # Executing the SQL query
    cursor.execute(sql_query)
    result = cursor.fetchone()

    if result:
        min_age_under_18 = result[0]
        max_age_under_18 = result[1]
        min_age_18_100 = result[2]
        max_age_18_100 = result[3]

        # Inserting records 
        if min_age_under_18 is not None and max_age_under_18 is not None:
            insert_query = """
                INSERT INTO Extra (age_group, min_age, max_age)
                VALUES ('Under 18', %s, %s);
            """
            cursor.execute(insert_query, (min_age_under_18, max_age_under_18))

        if min_age_18_100 is not None and max_age_18_100 is not None:
            insert_query = """
                INSERT INTO Extra(age_group, min_age, max_age)
                VALUES ('18-100', %s, %s);
            """
            cursor.execute(insert_query, (min_age_18_100, max_age_18_100))

        # Committing
        conn.commit()
        print("Records inserted successfully.")

    else:
        print("No data found for the specified conditions.")

except pymysql.Error as e:
    print("Error executing SQL query:", e)

Extra table created successfully.
Records inserted successfully.


**AVG**<br>
AVG() will take the average of a numeric field.

**Question:** What is the average age of the people in the data?
<br>

In [23]:
#Write a query to find the average of a numerical column.      

# Executing the SQL query 
query = """
    SELECT AVG(age) AS age
    FROM Person
    WHERE age IS NOT NULL;
"""

cursor.execute(query)
# Getting the results
result = cursor.fetchone()

# Printing the average age
print("Average age:", result[0])



Average age: 36.9808


Although this query gives us a quick answer, as a stand alone data frame, it is not that useful. We can add it to our longer query above.

In [72]:
#Add the AVG() aggregate function to your CASE query above that includes the MIN() and MAX()

try:
    # Connecting to database
    conn = pymysql.connect(host='localhost',
                           user='root',
                           password='1734680nN',
                           database='police_shootings')

    # Creating a cursor
    cursor = conn.cursor()

    # Defining the SQL query
    sql_query = """
        SELECT
            MIN(CASE WHEN age < 18 THEN age END) AS min_age_under_18,
            MAX(CASE WHEN age < 18 THEN age END) AS max_age_under_18,
            AVG(CASE WHEN age < 18 THEN age END) AS avg_age_under_18,
            MIN(CASE WHEN age >= 18 AND age <= 100 THEN age END) AS min_age_18_100,
            MAX(CASE WHEN age >= 18 AND age <= 100 THEN age END) AS max_age_18_100,
            AVG(CASE WHEN age >= 18 AND age <= 100 THEN age END) AS avg_age_18_100
        FROM
            Person;
    """

    # Executing the SQL query
    cursor.execute(sql_query)
    result = cursor.fetchone()

    if result:
        min_age_under_18 = result[0]
        max_age_under_18 = result[1]
        avg_age_under_18 = result[2]
        min_age_18_100 = result[3]
        max_age_18_100 = result[4]
        avg_age_18_100 = result[5]

        # Displaying the results
        print("Age statistics for individuals under 18:")
        print("Minimum Age:", min_age_under_18)
        print("Maximum Age:", max_age_under_18)
        print("Average Age:", avg_age_under_18)

        print("\nAge statistics for individuals aged 18-100:")
        print("Minimum Age:", min_age_18_100)
        print("Maximum Age:", max_age_18_100)
        print("Average Age:", avg_age_18_100)

    else:
        print("No data found for the specified conditions.")

except pymysql.Error as e:
    print("Error executing SQL query:", e)



Age statistics for individuals under 18:
Minimum Age: 2
Maximum Age: 17
Average Age: 15.9251

Age statistics for individuals aged 18-100:
Minimum Age: 18
Maximum Age: 92
Average Age: 37.3762


Now we are starting to get multiple values that we can save any time as a .csv if needed to access later.

In [78]:
#Save your new result set as .csv file!
import csv 

try:
    # Establishing a connection 
    conn = pymysql.connect(host='localhost',
                           user='root',
                           password='1734680nN',
                           database='police_shootings')

    # Creating a cursor
    cursor = conn.cursor()

    # Define the SQL query to fetch age statistics
    sql_query = """
        SELECT
            MIN(CASE WHEN age < 18 THEN age END) AS min_age_under_18,
            MAX(CASE WHEN age < 18 THEN age END) AS max_age_under_18,
            AVG(CASE WHEN age < 18 THEN age END) AS avg_age_under_18,
            MIN(CASE WHEN age >= 18 AND age <= 100 THEN age END) AS min_age_18_100,
            MAX(CASE WHEN age >= 18 AND age <= 100 THEN age END) AS max_age_18_100,
            AVG(CASE WHEN age >= 18 AND age <= 100 THEN age END) AS avg_age_18_100
        FROM
            Person;
    """

    # Executing the SQL query
    cursor.execute(sql_query)
    result = cursor.fetchone()

    if result:
        csv_file = 'age_statistics.csv' #CSV File creation

        # Writinf the query results to the CSV file
        with open(csv_file, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Statistic', 'Under 18', '18-100'])
            writer.writerow(['Minimum Age', result[0], result[3]])
            writer.writerow(['Maximum Age', result[1], result[4]])
            writer.writerow(['Average Age', result[2], result[5]])

        print(f"Age statistics saved to '{csv_file}' successfully.")

    else:
        print("No data found for the specified conditions.")

except pymysql.Error as e:
    print("Error executing SQL query:", e)


Age statistics saved to 'age_statistics.csv' successfully.


**Aliases (AS)**
<br>
You can abbreviate your code to make it more visually appealing...or more confusing? :) <br>
<br>
Examples:<br>
FROM table_name t<br>
FROM table_name AS t<br>
<br>
**Objective:** Use abbreviations or aliases for all tables for the same code you wrote above. Be sure to obtain the same result set.

In [90]:
#Rewrite your CASE/MIN/MAX/AVG code to include aliases if you have not done so already:

try:
    # Connection to your MySQL database
    conn = pymysql.connect(host='localhost',
                           user='root',
                           password='1734680nN',
                           database='police_shootings')

    # Creating a cursor 
    cursor = conn.cursor()

    # Defining the SQL query with table alias
    sql_query = """
        SELECT *
        FROM Person AS p
        LIMIT 5
    """

    # Executing the SQL query
    cursor.execute(sql_query)
    rows = cursor.fetchall()

    # Printing fetched rows to see if it worked
    for row in rows:
        print(row)

except pymysql.Error as e:
    print(f"Error executing SQL query: {e}")

(1, 'Tim Elliot', 53, 'M', 'A')
(2, 'Lewis Lee Lembke', 47, 'M', 'W')
(3, 'John Paul Quintero', 23, 'M', 'H')
(4, 'Matthew Hoffman', 32, 'M', 'W')
(5, 'Michael Rodriguez', 39, 'M', 'H')


**DELETE** ~Caution!~
<br>
You can delete all records from specific tables or set a criteria to delete certain values or NULL values without deleting the table itself. It is okay if you do not execute the code if you have completed all data cleaning steps earlier in the semester.<br>
<br>
If you created autoincrement IDs for any of your data, it is recommended to use TRUNCATE TABLE instead, used the same way. The ID's will automatically reset if needed.<br>
<br>
**Example Objective:** Delete all name data from the Person table.
<br>
**What table(s) are we deleting records from?** Person

In [11]:
#Replace the above info with your data. 
#You can write the code, but do not execute if you do not want to delete any of your data.

# Database connection details
host = 'localhost'
user = 'root'
password = '1734680nN'
database = 'police_shootings'

try:
    # Establish a connection to the MySQL database
    conn = pymysql.connect(host=host, user=user, password=password, database=database)
    cursor = conn.cursor()

    # Define the SQL query to delete rows from the Person table based on name
    name_to_delete = 'John Doe'  # Specify the name to delete
    sql_query = f"""
        DELETE FROM Person
        WHERE name = '{name_to_delete}';
    """

    # Print the SQL query for debugging
    print(f"Executing SQL query: {sql_query}")

    # Execute the SQL query to delete rows
    cursor.execute(sql_query)

    # Commit the changes to the database
    conn.commit()

    # Print a success message
    print(f"Rows with name '{name_to_delete}' deleted successfully.")

except pymysql.Error as e:
    # Handle any errors that occur during execution
    print(f"Error: {e}")


We can also delete entire tables in MySQL workbench by manually right clicking on the table and DROP TABLE. <br>
MySQL Workbench will prompt you to review the SQL syntax before dropping the table.<br>
The syntax is simple:<br>
DROP TABLE table_name<br>
<br>
We can also use dbExecute(con, DROP TABLE) from Jupyter Notebook/Rmarkdown/Visual Studio. You do not have to execute this code in this notebook if you do not want to delete a table.

In [150]:
#Close the database connection :)
cursor.close()
conn.close()

**STOP**<br>
Before you submit, did you comment all your code?<br>
Did you answer all of the questions in the markdown cells?<br>
Did you rename the file and write your name at the top of the .pynb?<br>
Attach the .csv file you created with your Blackboard submission. 